In [76]:
import pandas as pd
import numpy as np

In [68]:
# Load school data
schools = pd.read_csv('schools.csv')

schools['idx'] = schools.index

print(f"Loaded {len(schools)} school locations.")
schools.head()

Loaded 661 school locations.


,short_name,governance,address,commarea,zip,lat,long,grade_cat,idx
0,GLOBAL CITIZENSHIP,Charter,4647 W 47TH ST,GARFIELD RIDGE,60632,41.807579,-87.740097,ES,0
1,ACE TECH HS,Charter,5410 S STATE ST,WASHINGTON PARK,60609,41.796122,-87.625849,HS,1
2,LOCKE A,Charter,3141 W JACKSON BLVD,EAST GARFIELD PARK,60612,41.877248,-87.705235,ES,2
3,ASPIRA - EARLY COLLEGE HS,Charter,3986 W BARRY AVE,AVONDALE,60618,41.937298,-87.727096,HS,3
4,ASPIRA - HAUGAN,Charter,3729 W LELAND AVE,ALBANY PARK,60625,41.966406,-87.721825,ES,4


In [165]:
# Load crimes data
crimes = pd.read_csv('2018_crime.csv')
crimes = crimes.dropna()

#remove this block if desired
crimes = crimes.set_index('case_number')
crimes = crimes[['primary_type', 'latitude', 'longitude']]
#crimes = crimes.head(1000) #remove after testing

# add cols to populate later
crimes['nearest_school'] = ''
crimes['nearest_school_dist'] = ''

print(f"Loaded {len(crimes)} crime records.")
crimes.head()

Loaded 260405 crime records.


,primary_type,latitude,longitude,nearest_school,nearest_school_dist
case_number,,,,,
JB140548,HOMICIDE,41.680739,-87.645661,,
JB108189,MOTOR VEHICLE THEFT,41.994567,-87.659016,,
JB108139,MOTOR VEHICLE THEFT,42.012201,-87.693905,,
JB108319,THEFT,41.890384,-87.624110,,
JB108166,BURGLARY,41.717160,-87.643094,,


In [169]:
def get_closest_school(lat, long):
    distances = schools.apply(
        lambda row: 69.02 * ((lat - row['lat'])**2 + (long - row['long'])**2)**0.5,
        axis=1)
    i = distances.idxmin()
    #return schools.iloc[i]
    return [schools.iloc[i]['short_name'], schools.iloc[i]['idx'], distances[i]]
    
get_closest_school(41.8111828673, -87.5987862684)


['U OF C - NKO', 49, 0.0]

In [172]:

%%time
c = crimes.head(500)
#print(c.head())

#c.apply(lambda row: get_closest_school(41.8111828673, -87.5987862684), axis=1)

c['temp'] = c.apply(lambda row: get_closest_school(row['latitude'], row['longitude']), axis=1)

c[['nearest_school', 'nearest_school_index', 'nearest_school_dist']] = pd.DataFrame(c['temp'].values.tolist(), index=c.index)


c = c.drop(columns = ['temp'])

c














Wall time: 13.5 s


C:\Users\David Tabor\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\David Tabor\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\frame.py:3140: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


In [55]:


# Iterrows implementation
# for index, row in crimes.iterrows():
#     crime_lat = row['latitude']
#     crime_long = row['longitude']
#     dist_min = 1000

#     for i, r in schools.iterrows():
#         school_lat = r['lat']
#         school_long = r['long']
#         dist = 69.02 * ( (crime_lat - school_lat)**2 + (crime_long - school_long)**2   )**0.5
        
#         if (dist < dist_min): 
#             dist_min=dist
#             crimes.set_value(index, 'nearest_school', r['short_name'])
#             crimes.set_value(index, 'nearest_school_dist', dist_min)

# crimes.head()

In [54]:
#create full outer product -  BAD WAY TO DO IT

# schools['key'] = 1
# crimes['key'] = 1

# df = pd.merge(crimes, schools, on='zip')

# df.head()

# idxs = df.groupby('case_number')['dist_mi'].idxmin()
# df2 = df.loc[idxs]

# df2.head()

